In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
import httpx
import pandas as pd

from urllib.parse import urljoin

from will_it_saturate.servers import (
    FastAPIUvicornServer,
    NginxDockerServer,
    DjangoGunicornWSGIServer,
)
from will_it_saturate.epochs import Epoch
from will_it_saturate.hosts import Host, HostDetails

from will_it_saturate.results import Result
from will_it_saturate.files import BenchmarkFile
from will_it_saturate.control.client import ControlClient
from will_it_saturate.repositories import SqliteRepository, register_default_tables
from will_it_saturate.clients import (
    HttpxClient,
    AioHttpClient,
    WrkClient,
    BaseClient,
    ClientParameters,
)

<IPython.core.display.Javascript object>

# Remote Benchmark

In [ ]:
# dont_test

# create server control client + server (needed for turning files into urls)
server_control_host = Host(name="macmini.fritz.box", port=8001)
server_control_client = ControlClient(host=server_control_host)
server_details = server_control_client.get_host_details()
server = server_control_client.get_or_create_server(FastAPIUvicornServer())
print(server)

# create files and urls
byte, file_size, duration = 8, 10 ** 7, 3
epoch = Epoch(file_size=file_size, duration=duration)
files = server_control_client.get_or_create_files(epoch)
print(files[:10])
epoch.urls = [server.file_to_url(file) for file in files]

# create control client + measure server
client_control_host = Host(name="localhost", port=8001)
client_control_client = ControlClient(host=client_control_host)
client_details = client_control_client.get_host_details()
benchmark_client = HttpxClient(name="httpx")
client_params = ClientParameters(**benchmark_client.params())


result = Result(
    server=server,
    client=benchmark_client,
    server_details=server_details,
    client_details=client_details,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

protocol='http' name='fastAPI/uvicorn' host='localhost' port=5001


HTTPStatusError: 500 Server Error: Internal Server Error for url: http://macmini.fritz.box:8001/epochs/
For more information check: https://httpstatuses.com/500

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

result.elapsed = client_control_client.measure(client_params, epoch)
print(result.elapsed)

# result.elapsed = benchmark_client.measure(epoch)

In [ ]:
# dont_test

result.bytes_per_second

# Save Result

In [ ]:
# dont_test

repository = SqliteRepository.build_repository("test.db")
register_default_tables(repository)
results = repository.tables["result"]

In [ ]:
# dont_test

result_id = results.add_result(result)
print(result_id)

In [ ]:
# dont_test

results.get_results()

In [ ]:
# dont_test

pd.DataFrame([r.dict_for_pandas() for r in results.get_results()])